In [22]:
from pyhive import presto
from pandas import DataFrame
import numpy as np
import pandas as pd

# Presto Connection

In [25]:
presto_conn = presto.connect(
    host='165.232.118.33',
    port=8090,
    catalog='mongodb',
    schema='tbdmproject'
)
presto_cur = presto_conn.cursor()

# Creazione Tabella per controllo delle misure salvate

In [26]:
presto_cur.execute("CREATE TABLE IF NOT EXISTS checked (uuid varchar)")
presto_cur.fetchall()

[(True,)]

# Creazione Tabella delle Misure

In [27]:
presto_cur.execute("CREATE TABLE IF NOT EXISTS firstapprmeasures (t bigint,d varchar,u varchar,tz varchar,v double,k varchar, uuid varchar)")
presto_cur.fetchall()

[(True,)]

# Creazione Dataframe con explode delle measures

In [28]:
presto_cur.execute("SELECT a.* FROM iotsimulator a LEFT JOIN checked b ON a.uuid=b.uuid WHERE a.uuid IS NULL OR b.uuid IS NULL ORDER BY tz")
records = presto_cur.fetchall()
df = DataFrame(records)
df.columns = ['ref', 'cuid', 't','tz','cat','sn','type','uuid','m']
df=df.explode('m')
#elimina righe superflue
df.dropna(subset=['m'], inplace=True)
#controlla quante righe con m=null sono rimaste
df['m'].isnull().sum()
df=pd.concat([df.m.apply(pd.Series), df.drop('m', axis=1)], axis=1)
#sr=df.m.apply(pd.Series)
df.rename(columns={0: 't', 1: 'd', 2: 'u',3: 'mtz',4: 'v',5: 'k'}, inplace=True)
#dataframe finale
df

,t,d,u,mtz,v,k,ref,cuid,t,tz,cat,sn,type,uuid
0,1685378560100,None,℃,2023-05-29T16:42:40.100Z,2.1481,device_temperature,jzp://edv#0504.0000,e283102c-cd6d-4981-915c-0ba1623fa1c6,1685378560099,2023-05-29T16:42:40.099Z,0600,153,environmental,5c772298-53a3-4b2b-abfe-7ed24d833799
0,1685378560100,None,V,2023-05-29T16:42:40.100Z,3.3345,battery_level,jzp://edv#0504.0000,e283102c-cd6d-4981-915c-0ba1623fa1c6,1685378560099,2023-05-29T16:42:40.099Z,0600,153,environmental,5c772298-53a3-4b2b-abfe-7ed24d833799
0,1685378560100,None,℃,2023-05-29T16:42:40.100Z,21.4322,dew_point,jzp://edv#0504.0000,e283102c-cd6d-4981-915c-0ba1623fa1c6,1685378560099,2023-05-29T16:42:40.099Z,0600,153,environmental,5c772298-53a3-4b2b-abfe-7ed24d833799
0,1685378560100,None,%,2023-05-29T16:42:40.101Z,80.0000,relative_humidity,jzp://edv#0504.0000,e283102c-cd6d-4981-915c-0ba1623fa1c6,1685378560099,2023-05-29T16:42:40.099Z,0600,153,environmental,5c772298-53a3-4b2b-abfe-7ed24d833799
0,1685378560101,None,,2023-05-29T16:42:40.101Z,0.0000,orientation,jzp://edv#0504.0000,e283102c-cd6d-4981-915c-0ba1623fa1c6,1685378560099,2023-05-29T16:42:40.099Z,0600,153,environmental,5c772298-53a3-4b2b-abfe-7ed24d833799
0,1685378560101,None,℃,2023-05-29T16:42:40.101Z,37.0000,environmental_temperature,jzp://edv#0504.0000,e283102c-cd6d-4981-915c-0ba1623fa1c6,1685378560099,2023-05-29T16:42:40.099Z,0600,153,environmental,5c772298-53a3-4b2b-abfe-7ed24d833799
0,1685378560101,None,,2023-05-29T16:42:40.101Z,0.0000,back_front,jzp://edv#0504.0000,e283102c-cd6d-4981-915c-0ba1623fa1c6,1685378560099,2023-05-29T16:42:40.099Z,0600,153,environmental,5c772298-53a3-4b2b-abfe-7ed24d833799
0,1685378560101,None,,2023-05-29T16:42:40.101Z,0.0000,regular,jzp://edv#0504.0000,e283102c-cd6d-4981-915c-0ba1623fa1c6,1685378560099,2023-05-29T16:42:40.099Z,0600,153,environmental,5c772298-53a3-4b2b-abfe-7ed24d833799
0,1685378560101,None,,2023-05-29T16:42:40.101Z,0.0000,movement_level,jzp://edv#0504.0000,e283102c-cd6d-4981-915c-0ba1623fa1c6,1685378560099,2023-05-29T16:42:40.099Z,0600,153,environmental,5c772298-53a3-4b2b-abfe-7ed24d833799
0,1685378560101,None,,2023-05-29T16:42:40.101Z,0.0000,posture_tilt,jzp://edv#0504.0000,e283102c-cd6d-4981-915c-0ba1623fa1c6,1685378560099,2023-05-29T16:42:40.099Z,0600,153,environmental,5c772298-53a3-4b2b-abfe-7ed24d833799


# Perform insert into inside measures table

In [29]:
df_measures = df.copy()
df_measures = df_measures.drop(columns=['cat', 'cuid', 'mtz', 'sn', 'type', 'ref'])
df_measures = df_measures.T.groupby(level=0).first().T
data=[tuple(x) for x in df_measures.to_numpy()]
sql='INSERT INTO firstapprmeasures (d, k, t, tz, u, uuid, v) VALUES (%s, %s, %s, %s, %s, %s, %s)'
presto_cur.fast_executemany=True
presto_cur.executemany(sql, data)
presto_conn.commit()

# Perform insert into inside checked table

In [30]:
df_checked = df.copy()
df_checked = df_checked.drop(columns=['cat', 'cuid', 'mtz', 'sn', 'type', 'ref','d', 'k', 't', 'tz', 'u', 'v'])
df_checked = df_checked.T.groupby(level=0).first().T
data2=[tuple(x) for x in df_checked.to_numpy()]
sql='INSERT INTO checked (uuid) VALUES (%s)'
presto_cur.executemany(sql, data2)
presto_conn.commit()